In [10]:
from Crypto.Cipher import AES, PKCS1_OAEP
from Crypto.PublicKey import RSA
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad
from PIL import Image
import numpy as np
import pywt

def aes_rsa_dwt():
    # AES Encryption
    def encrypt_aes(text, key):
        cipher = AES.new(key, AES.MODE_CBC)
        cipher_text = cipher.encrypt(pad(text.encode(), AES.block_size))
        return cipher_text, cipher.iv

    # RSA Encryption
    def encrypt_rsa(text, public_key):
        cipher_rsa = PKCS1_OAEP.new(public_key)
        cipher_text = cipher_rsa.encrypt(text.encode())
        return cipher_text

    # LSB Embedding
    def embed_LSB(image, data):
        binary_data = ''.join(format(byte, '08b') for byte in data)
        data_index = 0
        height, width = image.shape[:2]  # Get image dimensions

        for y in range(height):
            for x in range(width):
                pixel = format(image[y, x], '08b')
                pixel = pixel[:-1] + binary_data[data_index]
                image[y, x] = int(pixel, 2)
                data_index += 1
                if data_index == len(binary_data):
                    break
            if data_index == len(binary_data):
                break

        return image


    # Load Image
    input_image_path = 'C:\\Users\\ashna\\Downloads\\input_image.png'
    image = np.array(Image.open(input_image_path))

    # Text to be encrypted and embedded
    text = 'hello'

    # AES Encryption
    aes_key = get_random_bytes(32)  # AES-256 key
    cipher_text_aes, iv_aes = encrypt_aes(text, aes_key)

    # RSA Encryption
    rsa_key = RSA.generate(2048)  # Generate RSA key pair
    public_key = rsa_key.publickey()
    cipher_text_rsa = encrypt_rsa(text, public_key)

    # DWT on image
    coeffs = pywt.dwt2(image.astype(float), 'haar')  # Ensure image data is in float format
    cA, (cH, cV, cD) = coeffs

    # Scale the cA coefficients to the range of 0-255 for embedding
    cA_scaled = np.clip(cA, 0, 255).astype(np.uint8)

    # Embed cipher_text_aes into the cA coefficient using LSB
    cA_embedded_aes = embed_LSB(cA_scaled.copy(), cipher_text_aes)

    # Embed cipher_text_rsa into the cA coefficient using LSB
    cA_embedded_rsa = embed_LSB(cA_scaled.copy(), cipher_text_rsa)

    # Reconstruct image from modified coefficients
    modified_coeffs_aes = (cA_embedded_aes, (cH, cV, cD))
    modified_image_aes = pywt.idwt2(modified_coeffs_aes, 'haar')

    modified_coeffs_rsa = (cA_embedded_rsa, (cH, cV, cD))
    modified_image_rsa = pywt.idwt2(modified_coeffs_rsa, 'haar')

    # Convert reconstructed images back to uint8 format
    modified_image_aes = np.clip(modified_image_aes, 0, 255).astype(np.uint8)
    modified_image_rsa = np.clip(modified_image_rsa, 0, 255).astype(np.uint8)

    # Save images
    cv2.imwrite('C:\\Users\\ashna\\Downloads\\aes-dwt-output.png', modified_image_aes)
    cv2.imwrite('C:\\Users\\ashna\\Downloads\\rsa-dwt-output.png', modified_image_rsa)